# Parser 3.00 - Arithmetic Negation

The goal for this version of the parser is to be able to recognize arithmetic negation.

On the surface this may not seem an obvious next step. That might instead seem to be subtraction, but in fact the ability to create a negative number by means other than subtraction is necessary in order to thoroughly test subtraction. Without this ability the equal precedence of subtraction and addition would prevent us from ever creating two different negative numbers in the same expression (try it). So we could never subtract a negative from a negative, and our tests would be incomplete.

>We could instead introduce grouping parentheses at this point to alter the order of evaluation. This would allow us to create two different negative numbers by indirect means, also solving the problem. However it seems to make more sense to introduce negative numbers first.

Why introduce a negation operator instead of making the parser capable of directly recognizing negative literal values? Mainly because eventually we will want to apply negation to variables as well as literals, so it seems a more general solution.

There are four characteristics of the unary negation operator **'-'** that we need to consider:

- it is [unary](https://en.wikipedia.org/wiki/Unary_operation)


- it is prefix


- it is [right associative](https://en.wikipedia.org/wiki/Operator_associativity)


- it has a [higher precedence](https://en.wikipedia.org/wiki/Order_of_operations) than binary addition

Arithmetic negation happens to a single operand, and the operator appears before the operand (as opposed to appearing after the operand, or as a suffix). Right association requires that when two or more negation operators are applied to the same operand, they are performed right to left. Higher precedence means that if both our operators are applied to the same operand, negation happens before addition.

>The idea that a computer programming language should interpret mathematical expressions the same way we all learned in grade school is common but not universal. [APL](https://en.wikipedia.org/wiki/APL_(programming_language)), [Forth](https://en.wikipedia.org/wiki/Forth_(programming_language)) and [Lisp](https://en.wikipedia.org/wiki/Forth_(programming_language)) are some of the languages which evaluate expressions using rules which do not include operator precedence. However we will not go down that path here.

Now that we are going to have two operators of unequal precedence, we need a way to parse expressions that might contain any combination of them. For this we will adopt (and eventually adapt) [Dijkstra’s](https://en.wikipedia.org/wiki/Edsger_W._Dijkstra) [Shunting Yard algorithm](https://en.wikipedia.org/wiki/Shunting-yard_algorithm) for converting [infix expressions](https://en.wikipedia.org/wiki/Infix_notation) to [postfix](https://en.wikipedia.org/wiki/Reverse_Polish_notation).

Postfix is also commonly known as Reverse Polish notation (RPN). The advantage of RPN expressions is that they are easy to evaluate, left to right, with no ambiguity.

Why the Shunting Yard? Mainly because it made immediate sense to me when I first encountered it, which is not an experience I’ve had with [other parsing algorithms](https://en.wikipedia.org/wiki/Top-down_parsing).


## Libraries

In [ ]:
import glob       # for searching directories

import re         # for regular exprssions

## User output

In [ ]:
visSep = '-------------'             # visual separator

def UIwriteln(this):
    '''write a single line to output'''
    print( f'{this}\n' )
    
def UIwriteSep():
    '''write a visual separator'''
    UIwriteln( visSep )

def UIshow(tag, value):
    '''write a tagged value to output'''
    UIwriteln( f'{tag}: {value}' )

def UIerror(this):
    '''write an error message to output'''
    UIshow( 'Error', this )

# Tracing

In [ ]:
# flags: show trace of processing

showInteract = True          # default for interactive use
showBatch = False            # default for batch use

showTrace = None             # control flag

# Trace Output

def TOshow(mesg, text):
    '''write trace message to output if enabled'''
    if showTrace:
        UIshow( f'{mesg:15s}', text )
        
def TOstring(tag, this):
    
    if showTrace:
        TOshow( tag, ' '.join([str(e) for e in this]) )

# -----------------------
# Parse Tracing
# -----------------------

def PTshowexpr(this):

    TOshow( 'Parse', visSep )
    TOshow( 'Current Expr', this )

def PTshowparse(ok, res, stk):

    if ok:
        TOstring( 'Current RPN', res )
        TOstring( 'Operator Stack', stk )

def PTshowtoken(this):

    if not this[0] == ' ':
        TOshow( "Found Token", this )

# -----------------------
# Evaluation Tracing
# -----------------------

def ETshowtoken(this):
    
    TOshow( 'Eval', visSep )
    TOshow( 'Current token', this )

def ETshoweval(ok, stk):
    
    if ok:
        TOstring( 'Operand Stack', stk )


As we are adding stack operations to both the parser and evaluator, we've added ways to keep an eye on them.

# Common

In [ ]:
intMax =  4294967295                # 2**32-1, for range checking
intMin = -4294967296                # -(2**32)

Previously we had no need to specify a lower limit for our integers. It was simply implied to be zero, as there was no way to create any smaller value. By adding signed integer values we perforce need to add a lower limit on them in order to play with out of range arithmetic results.

We have a slight problem in that tests already written used unsigned 32-bit values. Every bit is interpreted as a power of two. In particular, the most significant bit is interpreted as a magnitude just like all the other bits (zero if zero, 2\*\*31 if one).

In contrast, when interpreted as signed the most significant bit is used as a sign indicator (positive if zero, negative if one).

The count of representable values is the same for both signed and unsigned numbers, 2\*\*32, but for signed the positive maximum is half that of unsigned, 2\*\*31-1, or 2,147,483,647. Which means that some of our earlier tests will fail in regression if we keep to a 32-bit size for our numbers.

Which we do not have to do. The only limit on the [maximum integer value Python 3 can handle](https://docs.python.org/3/whatsnew/3.0.html#integers) is the underlying size of computer memory. It's a hardware limit, not a software one. Because we are interested in limits only so we can test them, we can simply use what are effectively 33-bit integer values. We rename *uintMax* to *intMax*, making our maximum postive value 2\*\*32-1. We introduce a minimum negative integer value of -2\*\*32.

>Note that the absolute value of *intMax* is one less than the absolute value of *intMin*. There are actually just as many positive as negative integers because zero is considered positive.

In **M** bit [two's complement](https://en.wikipedia.org/wiki/Two%27s_complement) representation, both zero (represented as **M** zero bits) and the smallest negative number (represented as a one bit followed by **M-1** zero bits) are their own negations. That is, negating them by flipping all **M** bits, adding one, and truncating to **M** bits results in their original values.

This is normally the expected result for zero, but not for the smallest negative number. We would expect that the result would be *intMax+1*, which is not possible to represent (we are pretending). We could simply ignore what happens and let the user deal with problem. Here we choose to make it illegal to negate *intMin*.

# Parser

In [ ]:
versionNumber = '3.00'

# operands accepted:
# - decimal integer literals
# - hexadecimal integer literals

# operators accepted:
# - unary negation
# - binary addition

# errors detected:
# - unrecognized input
# - out of range numeric input
# - malformed expression

# result tuple:
# - (True, [parse])
# - (False, None)

def PEdoparse(this):
       
    # initialize
    
    expr = this                # save to new variable but retain original for error reports
    start = 15                 # tracked so we can report where in an expression an error occurred
    token = None               # anything successfully matched
    ok = wantoperand = True    # flags
    result = []                # rpn expression
    stk = [ ('EOE', 1) ]       # operator stack
               
    def parseErr(mesg):
        '''report parse error'''
        UIerror(mesg)
        UIwriteln(f'>>> {this}')
        UIwriteln(f'{"^^near here".rjust(start)}')
        return False
     
    def popGEop(prec):
        '''pop operators of equal or greater precedence'''
        while prec <= stk[-1][1]:
            result.append(stk.pop()[0])
            
    def pushLeft(op, prec):
        '''push left associative operator on stack'''
        popGEop(prec)
        stk.append( (op, prec) )
            
    def popGop(prec):
        '''pop operators of greater precedence'''
        while prec < stk[-1][1]:
            result.append(stk.pop()[0])
            
    def pushRight(op, prec):
        '''push right associative operator on stack'''
        popGop(prec)
        stk.append( (op, prec) )
            
    # convert unsigned literal to internal form
    # - all chars in input known to be legal hexadecimal characters
    # - checks that value is within range
    
    def convertUint(ulit, base):
        
        uint = 0
        
        # isolate the significant portion of 'ulit'
        # - this drops any leading prefix and all leading zeroes
        # - if search fails, then input is all zeroes (and so is value)
        
        p = re.search('[1-9A-F][0-9A-F]*', ulit.upper())
        
        if p != None:
            for digit in p.group():
                digval = '0123456789ABCDEF'.find(digit)
                if uint <= (intMax - digval)/base:
                    uint =  uint * base + digval
                else:
                    return parseErr(f'\'{ulit}\' is out of range')
        
        result.append(uint)
        return True
    
    # test if expression starts with given regular expression
    
    def startsWith(regex):
        
        nonlocal expr, start, token
        
        p = re.match(regex, expr)
        if p == None:
            return False
        else:
            token = p.group()              # what we matched
            start += len(token)            # update to next match position in original string
            expr = expr[len(token):]       # "chop off" what we matched
            PTshowtoken(token)             # trace
            return True
                 
    # top level main loop
    
    while ok and len(expr):
        
        # skip leading whitespace
        
        _ = startsWith('[ ]+')
            
        # trace
        
        PTshowexpr(expr)
            
        # look for operand
             
        if wantoperand:
            
            if startsWith('[-]'):
                '''unary negation ?'''
                pushRight( token, 80 )
                
            else:
        
                wantoperand = False         # no non-state changing token found
            
                if startsWith('0[xX][0-9a-fA-F]+'):
                    '''unsigned hexadecimal literal ?'''
                    ok = convertUint(token, 16)
    
                elif startsWith('[0-9]+'):
                    '''unsigned decimal literal ?'''
                    ok = convertUint(token, 10)
    
                else:
                    '''malformed'''
                    ok = parseErr('Expecting operand')
            
        # look for operator
        
        else:
            
            wantoperand = True
            
            if startsWith('[+]'):
                '''binary addition ?'''
                pushLeft( token, 60 )
                
            else:
                '''malformed'''
                ok = parseErr('Expecting operator')
            
        # trace
        
        PTshowparse(ok, result, stk )
        
    # at end must be in 'wantoperator' state
    # - ie., last token must be an operand
                    
    if ok and wantoperand:
        ok = parseErr('Unexpected end of expression')
        
    # clear operator stack
    
    if ok:
        popGEop( 3 )
                    
    # done
                    
    return (ok, result if ok else None)

### How it works

We implement all of the basic mechanics of the Shunting Yard algorithm.

The parser relies on the use of a [stack](https://en.wikipedia.org/wiki/Stack_(abstract_data_type)) and its two fundamental operations of pushing and popping. Here it essentially functions as a holding area for operators we find in the infix expression. Once pushed on the stack, operators stay there until popped off.

In the *wantoperand* state, if we find an integer literal we immediately add it to the RPN expression we're building up. Then we switch to the *wantoperator* state.

If instead we find a negation operator, we push it. But first we pop off any higher precedence operators already on the stack and add them the RPN expression. Then we push the negation operator on the stack. We do not change the state, which allows us to write two or more negation operators in a row if we like.

>In this version such multiple negations can be written without separating the operators. However when the predecrement operator **'--'** is introduced, it will become necessary to put one or more spaces between them to guarantee multiple negation is what happens.

We initialize the operator stack with an *End of Expression* operator which has the lowest precedence of all. This acts as a backstop, making sure that we will always find a lower precedence operator.

>This operator never appears in either the original infix expression or the final RPN result. It is therefore sometimes called an *imaginary* operator. In a sense the only reason it's called an operator at all is that it appears on the operator stack. It doesn't actually have any effect on any operand.

In the *wantoperator* state we're still only looking for addition operators. First we switch states, because it's what we need to do if successful and it doesn't matter if we're not. If we are successful we do almost the same as for negation operators, except this time we pop operators with a greater or equal precedence.

>In the Shunting Yard algorithm, right associative operators like negation pop only higher precedence operators. Left associative operators like addition pop both higher and equal precedence operators.

Once there is no more expression to parse, we use a very low precedence to pop any operators still on the stack into the RPN. This is on purpose lower than that of any operator that can appear in the RPN, but still higher than the *End of Expression* operator.

Since operator precedence parsing relies only on relative values (higher precedences have higher values), their actual values are somewhat arbitrary. Here we are using values that leave room for adding operators with other precedences as we go along.

# Evaluator

In [ ]:
# operators handled:
# - unary negation
# - binary addition

# errors detected:
# - out of range

# return tuple:
# - (True, result)
# - (False, None)

def EEdoeval(rpn):
    
    stk = []
    ok = True
    
    def inRange(ok, val):
        '''range check test result'''
        if ok:
            stk.append( val )
        else:
            UIerror( 'Evaluation result out of range' )
        return ok
    
    def unNeg():
        '''unary negation'''
        arg = stk.pop()
        return inRange( arg != intMin, -arg )                   # watch for the un-negatable
            
    def binAdd():
        '''binary addition'''
        rgt = stk.pop()
        lft = stk.pop()
        
        # if left operand is positive:
        # required: lft + rgt <= intMax
        
        if lft >= 0:
            return inRange( rgt <= intMax - lft, lft+rgt )      # re-arranged to avoid overflow
                
        # if left operand is negative:
        # required: lft + rgt >= intMin
        
        else:
            return inRange( rgt >= intMin - lft, lft+rgt )      # re-arranged to avoid underflow
            
    # main loop
        
    for v in rpn:
        
        ETshowtoken(v)
        
        if v == '-':          # unary negation ?
            ok = unNeg()
            
        elif v == '+':        # binary addition ?
            ok = binAdd()
            
        else:                 # it's an operand
            stk.append( v )
            
        if not ok:
            return (False, None)
         
        ETshoweval( ok, stk )
            
    return ( True, stk.pop() )


### How it works

The evaluator also relies on the use of a stack. Unlike the parser, this stack holds operands. Operators pop one or two off, evaluate them, and push the result back on the stack. This continues until there is no more RPN to process. At this point there is only item on the stack, the final result

We do not need to explicitly check that things work out this way. A successful parse guarantees that the expression is properly formed, and a properly formed expression *will* work out this way. Of course it is possible to feed the evaluator deliberately malformed input, but we will not be doing that here and so will not guard against it.

Execution of binary operators pull operands off the stack in reverse of the order they were pushed on, hence the right-hand operand comes off first. This is not so important for addition, which is commutative, but it will be for operators which are not.

For simplicity we perform both a range check of the result and also the addition. This lets us use the single function *inRange()* to both check the result and update the operand stack. If the result is out of range, we discard the result and report an error.

>We could push the result in any case, since the whole stack is going to be discarded after an error anyway. This is a matter of preference; either way works just fine.

The introduction of signed values and a minimum allowed negative integer means that range checking becomes more complicated.

There are four possible combinations of operand signs:

1. positive + positive: greater than *intMax* possible
2. positive + negative: out of range not possible
3. negative + positive: out of range not possible
4. negative + negative: less than *intMin* possible

Based on these combinations we might write a range check like this:

```python
# lft and rgt positive ?
# - req: lft + rgt <= intMax

if lft >= 0 and rgt >= 0:
    ok = rgt <= intMax - lft          # re-arranged to avoid overflow

# lft and rgt negative ?
# req: lft + rgt >= intMin

elif lft < 0 and rgt < 0:
    ok = rgt >= intMin - lft          # re-arranged to avoid underflow

# signs differ - no out of range possible

else:
    ok = 1
```

The above code has the apparent advantage that a check is performed only in cases where out of range results are actually possible. So why does the code in the evaluation loop always perform a check? It always gives the same result, but is it actually an improvement?

Maybe. We note that despite any [short circuit evaluation](https://en.wikipedia.org/wiki/Short-circuit_evaluation) of [Python's](https://www.python.org/) **and** logical-AND operator, the version above always performs at least three comparisons even when no check is ultimately made. If a check is made, there are four comparisons and one subtraction.

The version used always makes two comparisons and one subtraction. If the cost of comparison and subtraction are similar – and they should be, since at the hardware level a comparison is just a subtraction with the result value discarded – then overall the second version should be faster.

Does the function call in the version used make a difference to this analysis? No, because we would also make that same call if we used the above version, and for the same reason (to centralize and simplify range checking and stack pushing).


## Running the parser

In [ ]:
passCnt = failCnt = 0                       # most useful for test input files, but never any harm

def startUp(flag):
    '''begin execution'''
    global passCnt, failCnt, showTrace
    UIshow( 'Parser', versionNumber )
    passCnt = failCnt = 0
    showTrace = flag
    
def shutDown():
    '''terminate execution'''
    UIwriteSep()
    UIshow( 'Pass', passCnt )
    UIshow( 'Fail', failCnt )
    
# run parser

def parseOne(this):
    '''parse/evaluate one expression'''
    global passCnt, failCnt
    UIwriteSep()
    UIshow( 'Input', this )
    ok, res = PEdoparse( this )
    if ok:
        UIshow( 'Final Parse', res )
        ok, res = EEdoeval( res )
        if ok:
            UIshow( 'Final Eval', res )
    if ok:
        passCnt += 1
    else:
        failCnt += 1

## Interactive use

In [ ]:
def parse():
    
    startUp(showInteract)
    while True:
        inp = input( 'Expression: ' )
        UIwriteln( '' )                      # looks better with a blank line here
        if inp.upper()[0] == 'Q':
            break
        elif inp.strip():
            parseOne( inp )
    shutDown()

## Batch processing

In [ ]:
testDir = '..\\ParserTest\\'            # directory holding test input files (empty string if same as notebook directory)

# convert current version number to match test file numbers
# - done this way so we can update only the version number and everything still works

def currNum():
    
    head = versionNumber[:len(versionNumber)-3]
    tail = versionNumber[-2:]
    return f'{head:0>2}{tail}'

# make full path name to test file

def makePath(typ, num):
    return f'{testDir}{typ}{num}.txt'

# run one test

def runTest(this):
    
    UIwriteln(f'Parser {versionNumber} vs {this[-12:-4]}')
    
    with open(this) as f:
        data = f.readlines()
    for line in data:
        test = line.strip()
        if test and test[0] != '#':         # skip blank and comment lines
            parseOne(test)
    
# run a test of current or specified version which should succeed
    
def good(num='curr'):
  
    startUp(showBatch)
    runTest(makePath('pass', currNum() if num == 'curr' else num))
    shutDown()
    
# run a test of current or specified version which should fail

def bad(num='curr'):
    
    startUp(showBatch)
    runTest(makePath('fail', currNum() if num == 'curr' else num))
    shutDown()
    
# run regression test against current and all previous test files

def regress():
            
    UIwriteln('PASS tests')
    
    currFn = makePath('pass', currNum())

    startUp(showBatch)
    failed = []
    fnlist = glob.glob(f'{testDir}pass????.txt')
    for fn in fnlist:
        if fn <= currFn:
            atstart = failCnt
            runTest(fn)
            if atstart < failCnt:
                failed.append(fn)               
    shutDown()
    
    UIwriteln('FAIL tests')
    
    currFn = makePath('fail',currNum())

    startUp(showBatch)
    passed = []
    fnlist = glob.glob(f'{testDir}fail????.txt')
    for fn in fnlist:
        if fn <= currFn:
            atstart = passCnt
            runTest(fn)
            if atstart < passCnt:
                passed.append(fn)               
    shutDown()
    
    if not len(failed):
        UIwriteln('All pass tests succeded')
    else:
        UIwriteln('Pass tests which failed')
        for fn in failed:
            UIwriteln(f'  {fn}')
            
    if not len(passed):
        UIwriteln('All fail tests succeded')
    else:
        UIwriteln('Fail tests which passed')
        for fn in passed:
            UIwriteln(f'   {fn}')
              

# Testing the parser

In [ ]:
parse()       # interactive, one expression at a time

In [ ]:
good()        # run current parser against its own pass test. Use good('1234') to run against specific pass test.

In [ ]:
bad()         # run current parser against its own fail test. Use bad('5678') to run against specific fail test.

In [ ]:
regress()     # run parser against all previous and current tests